In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv

# Define parameters
pi = np.pi
p_G = 2.4E-6  # Grating period in meters
x_1 = 0
L_1 = 2  # meters
lam = 5E-10  # Distance between source and first grating in meters
L = 4  # meters, distance between last grating and camera
h = 15E-6
N = 5E28
b = 4.1491E-15
slit_size = 240E-6  # Slit size in meters
lambdas = [5E-10]  # List of wavelengths to evaluate

# Function to calculate contrast V and K_max for a given d and lambda
def calculate_contrast_s_and_kmax(d, lam):
    p_s = p_G
    k_0 = 2 * pi / lam
    k_G = 2 * pi / p_G
    k_s = 2 * pi / p_s
    L_2 = L - L_1 - d  # Total distance from source to camera
    pm = L * p_G / d  # Moiré period
    k_m = 2 * pi / pm  # Moiré wave vector
    M1 = 1 + d / L_1  # First magnification
    M2 = 1 + (L_2) / (L_1 + d)  # Second magnification without sample
    m_max = 5  # Maximum absolute value of m (odd)

    def generate_G1(m_max, k_G, x_1, h):
        G1 = []  # Initialize G1 as an empty list
        G1.append([0, np.cos(N * b * lam * h / 2)])
        m_values = [m for m in range(-m_max, m_max + 1) if m % 2 != 0]
        for m in m_values:
            wave_vector = m * k_G
            amplitude = (2 / (m * pi)) * np.sin(N * b * lam * h / 2) * np.exp(1j * m * k_G * x_1)
            G1.append([wave_vector, amplitude])
        return G1

    def custom_convolution(P1, G1):
        P2 = []
        for row in G1:
            wave_vector, amplitude = row
            result = (P1[0] + wave_vector, P1[1] * amplitude)
            P2.append(result)
        return P2

    def scale_wave_vectors(P2, M1):
        P2n = [(wave_vector / M1, amplitude) for wave_vector, amplitude in P2]
        return P2n

    def phaseterm(P2n, d, k_0, M1):
        P3 = [
            (
                wave_vector,
                np.exp(-1j * d * (wave_vector**2) / (2 * (k_0 / M1))) * amplitude
            )
            for wave_vector, amplitude in P2n
        ]
        return P3

    def custom_convolution2(P3, G1):
        P4 = []
        for p3_row in P3:
            wave_vector_p3, amplitude_p3 = p3_row
            for g1_row in G1:
                wave_vector_g1, amplitude_g1 = g1_row
                result2 = (wave_vector_p3 + wave_vector_g1, amplitude_p3 * amplitude_g1)
                P4.append(result2)
        return P4

    def scale_wave_vectors2(P4, M1, M2):
        P4n = [(wave_vector / M2, amplitude) for wave_vector, amplitude in P4]
        return P4n

    def phaseterm2(P4n, d, k_0, M1, M2):
        P5 = [
            (
                wave_vector,
                np.exp(-1j * L_2 * ((wave_vector**2) / (2 * (k_0 / (M1 * M2)))) / M1) * amplitude
            )
            for wave_vector, amplitude in P4n
        ]
        return P5

    def compute_P5_conj(P5):
        P5_conj = []
        for wave_vector, amplitude in P5:
            first_term_conj = -wave_vector
            second_term_conj = np.conj(amplitude)
            P5_conj.append((first_term_conj, second_term_conj))
        return P5_conj

    def H_convolution_4(P5, P5_conj):
        H = []
        for P5_row in P5:
            wave_vector_P5, amplitude_P5 = P5_row
            for P5_conj_row in P5_conj:
                wave_vector_P5_conj, amplitude_P5_conj = P5_conj_row
                result4 = (wave_vector_P5 + wave_vector_P5_conj, amplitude_P5 * amplitude_P5_conj)
                H.append(result4)
        return H

    G1 = generate_G1(m_max, k_G, x_1, h)
    P1 = [0, 1]  # Initial conditions for convolution
    P2 = custom_convolution(P1, G1)
    P2n = scale_wave_vectors(P2, M1)
    P3 = phaseterm(P2n, d, k_0, M1)
    P4 = custom_convolution2(P3, G1)
    P4n = scale_wave_vectors2(P4, M1, M2)
    P5 = phaseterm2(P4n, d, k_0, M1, M2)
    P5_conj = compute_P5_conj(P5)
    H = H_convolution_4(P5, P5_conj)

    def aggregate_amplitudes(H):
        wave_vector_sums = {}
        for wave_vector, amplitude in H:
            rounded_wave_vector = np.round(wave_vector,1)
            if rounded_wave_vector in wave_vector_sums:
                wave_vector_sums[rounded_wave_vector] += amplitude
            else:
                wave_vector_sums[rounded_wave_vector] = amplitude
        H_reduced = [(wave_vector, amplitude) for wave_vector, amplitude in wave_vector_sums.items()]
        return H_reduced

    H_reduced = aggregate_amplitudes(H)

    def find_max_amplitude_and_wave_vector(H_reduced, k_m):
        max_ampl = 0
        k_max = 0
        for wave_vector, amplitude in H_reduced:
            if 0.9 * k_m <= wave_vector <= 1.1 * k_m:
                if amplitude > max_ampl:
                    max_ampl = amplitude
                    k_max = wave_vector
        return max_ampl, k_max

    def find_H0(H_reduced):
        H_0 = 0
        for wave_vector, amplitude in H_reduced:
            if wave_vector == 0:
                H_0 = amplitude
                break
        return H_0

    Max_ampl, K_max = find_max_amplitude_and_wave_vector(H_reduced, k_m)
    H_0 = find_H0(H_reduced)

    sinc_km = np.sinc(k_m * slit_size / (2 * pi))
    sinc_H0 = np.sinc(H_0 * slit_size / (2 * pi)) if H_0 != 0 else 0

    V = np.abs(2 * Max_ampl * sinc_km / (H_0 * sinc_H0)) if H_0 != 0 else None
    return V, K_max

# Range for d
d_range = np.arange(0.001, 0.085, 0.001)  # Range for d values

# Create dictionary to store V for different d values
V_dict = {}

# Store results to avoid recalculating
results_dict = {lam: [calculate_contrast_s_and_kmax(d, lam) for d in d_range] for lam in lambdas}

# Store V in the dictionary
for lam in lambdas:
    V_dict[lam] = {d: calculate_contrast_s_and_kmax(d, lam)[0] for d in d_range}

# Filter out contrasts less than 1%
V_dict_filtered = {}
for lam in lambdas:
    V_dict_filtered[lam] = {d: V for d, V in V_dict[lam].items() if V is not None and V >= 0.01}

# Print the filtered V dictionary
print("Filtered V dictionary:")
for lam, d_values in V_dict_filtered.items():
    print(f"\nFor wavelength λ = {lam:.1e} m:")
    for d, V in d_values.items():
        print(f"d = {d:.4f} m, V = {V:.5f}")  # Rounded to 3 decimal places

# Save the filtered contrast results to a CSV file
filename = 'V_withoutsample_filtered.csv'
with open(filename, 'w', newline='') as csvfile:
    fieldnames = ['d (m)', 'Contrast V']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for lam, d_values in V_dict_filtered.items():
        for d, V in d_values.items():
            writer.writerow({'d (m)': d, 'Contrast V': round(V, 5)})  # Rounded to 5 decimal places

# Plot Contrast V vs d for different wavelengths
fig, ax1 = plt.subplots(figsize=(10, 6))

for lam in lambdas:
    d_values = list(V_dict_filtered[lam].keys())
    contrast_values = list(V_dict_filtered[lam].values())
    
    # Plot contrast vs d on the primary y-axis
    ax1.plot(d_values, contrast_values, marker='o', linestyle='-', label=f'Contrast V (λ={lam:.1e} m)')

# Add labels and legend
ax1.set_xlabel('d (meters)')
ax1.set_ylabel('Contrast V')
ax1.set_title('V_withoutsample-monochromatic')
ax1.legend(loc='upper right')
ax1.grid(True)

# Create a secondary y-axis for K_max
ax2 = ax1.twinx()

for lam in lambdas:
    # Retrieve results for current lambda
    contrast_values, kmax_values = zip(*results_dict[lam])
    
    # Plot K_max vs d on the secondary y-axis
    # ax2.plot(d_range, kmax_values, marker='s', linestyle='--', label=f'K_max (λ={lam:.1e} m)')

# Add labels and legend for the secondary y-axis
ax2.set_ylabel('K_max')
# ax2.legend(loc='upper left')

plt.show()
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
import numpy as np
import matplotlib.pyplot as plt
import csv

# Define parameters
pi = np.pi
alpha = pi / 2
p_G = 2.4E-6  # Grating period in meters
x_1 = 0
L_1 = 2  # meters
lam = 5E-10  # Wavelength in meters
L = 4  # meters  # Distance between last grating and camera, in meters
h = 15E-6
#h_s = 15E-6
N = 5.0E28
b = 4.1491E-15
L_s = L / 3

# Define slit size
slit_size = 240E-6  # Slit size in meters

# Define different wavelengths
lambdas = [5E-10]  # List of wavelengths to evaluate

# Function to calculate contrast V and K_max for a given d and lambda
def calculate_contrast_s_and_kmax(d, lam):
    p_s = p_G
    k_0 = 2 * pi / lam
    k_G = 2 * pi / p_G
    k_s = 2 * pi / p_s
    L_2s = L - L_1 - d - L_s
    pm = L * p_G / d  # Moiré period
    k_m = 2 * pi / pm  # Moiré wave vector
    M1 = 1 + d / L_1  # First magnification
    M2s = 1 + L_2s / (L_1 + d)  # Second magnification with sample
    M3 = 1 + L_s / (L_1 + d + L_2s)  # Third magnification with sample

    m_max = 5  # Maximum absolute value of m (odd)

    # Function definitions
    def generate_G1(m_max, k_G, x_1, alpha=np.pi / 4):
        G1 = []  # Initialize G1 as an empty list
        G1.append([0, np.cos(N * b * lam * h / 2)])
        m_values = [m for m in range(-m_max, m_max + 1) if m % 2 != 0]
        for m in m_values:
            wave_vector = m * k_G
            amplitude = (2 / (m * pi)) * np.sin(N * b * lam * h / 2) * np.exp(1j * m * k_G * x_1)
            G1.append([wave_vector, amplitude])
        return G1

    def custom_convolution(P1, G1):
        P2 = []
        for row in G1:
            wave_vector, amplitude = row
            result = (P1[0] + wave_vector, P1[1] * amplitude)
            P2.append(result)
        return P2

    def scale_wave_vectors(P2, M1):
        P2n = [(wave_vector / M1, amplitude) for wave_vector, amplitude in P2]
        return P2n

    def phaseterm(P2n, d, k_0, M1):
        P3 = [
            (
                wave_vector,
                np.exp(-1j * d * (wave_vector**2) / (2 * (k_0 / M1))) * amplitude
            )
            for wave_vector, amplitude in P2n
        ]
        return P3

    def custom_convolution2(P3, G1):
        P4 = []
        for p3_row in P3:
            wave_vector_p3, amplitude_p3 = p3_row
            for g1_row in G1:
                wave_vector_g1, amplitude_g1 = g1_row
                result2 = (wave_vector_p3 + wave_vector_g1, amplitude_p3 * amplitude_g1)
                P4.append(result2)
        return P4

    def scale_wave_vectors2(P4, M1, M2s):
        P4n = [(wave_vector / M2s, amplitude) for wave_vector, amplitude in P4]
        return P4n

    def phaseterm2(P4n, d, k_0, M1, M2s):
        P5 = [
            (
                wave_vector,
                np.exp(-1j * L_2s * ((wave_vector**2) / (2 * (k_0 / (M1 * M2s)))) / M1) * amplitude
            )
            for wave_vector, amplitude in P4n
        ]
        return P5

    def generate_Gs1(m_max, k_s, pi):
        Gs1 = []
        Gs1.append([0, np.cos((pi/2)/2)])
        m_values = [m for m in range(-m_max, m_max + 1) if m % 2 != 0]
        for m in m_values:
            wave_vector = m * k_G
            amplitude = (2 / (m * pi)) * np.sin((pi/2)/2)
            Gs1.append([wave_vector, amplitude])
        return Gs1

    def custom_convolution3(P5, Gs1):
        P6 = []
        for p5_row in P5:
            wave_vector_p5, amplitude_p5 = p5_row
            for gs1_row in Gs1:
                wave_vector_gs1, amplitude_gs1 = gs1_row
                result3 = (wave_vector_p5 + wave_vector_gs1, amplitude_p5 * amplitude_gs1)
                P6.append(result3)
        return P6

    def scale_wave_vectors3(P6, M1, M2s, M3):
        P6n = [(wave_vector / M3, amplitude) for wave_vector, amplitude in P6]
        return P6n

    def phaseterm3(P6n, d, k_0, M1, M2s, M3):
        P7 = [
            (
                wave_vector,
                np.exp(-1j * L_s * ((wave_vector**2) / (2 * (k_0 / (M1 * M2s * M3)))) /(M1 * M2s)) * amplitude
            )
            for wave_vector, amplitude in P6n
        ]
        return P7

    def compute_P7_conj(P7):
        P7_conj = []
        for wave_vector, amplitude in P7:
            first_term_conj = -wave_vector
            second_term_conj = np.conj(amplitude)
            P7_conj.append((first_term_conj, second_term_conj))
        return P7_conj

    def H_sample_convolution_4(P7, P7_conj):
        H_sample = []
        for P7_row in P7:
            wave_vector_P7, amplitude_P7 = P7_row
            for P7_conj_row in P7_conj:
                wave_vector_P7_conj, amplitude_P7_conj = P7_conj_row
                result4_sample = (wave_vector_P7 + wave_vector_P7_conj, amplitude_P7 * amplitude_P7_conj)
                H_sample.append(result4_sample)
        return H_sample

    # Generate G1
    G1 = generate_G1(m_max, k_G, x_1)

    # Perform custom convolution to get P2
    P1 = [0, 1]  # Initial conditions for convolution
    P2 = custom_convolution(P1, G1)

    # Scale wave vectors to get P2n
    P2n = scale_wave_vectors(P2, M1)

    # Apply phase term to get P3
    P3 = phaseterm(P2n, d, k_0, M1)

    # Perform second convolution to get P4
    P4 = custom_convolution2(P3, G1)

    # Scale wave vectors to get P4n
    P4n = scale_wave_vectors2(P4, M1, M2s)

    # Apply phase term to get P5
    P5 = phaseterm2(P4n, d, k_0, M1, M2s)

    # Perform third convolution to get P6
    Gs1 = generate_Gs1(m_max, k_s, pi)

    P6 = custom_convolution3(P5, Gs1)

    # Scale wave vectors to get P6n
    P6n = scale_wave_vectors3(P6, M1, M2s, M3)

    # Apply phase term to get P7
    P7 = phaseterm3(P6n, d, k_0, M1, M2s, M3)

    # Compute conjugate of P7
    P7_conj = compute_P7_conj(P7)

    # Perform convolution to get H
    H = H_sample_convolution_4(P7, P7_conj)

    # Aggregate similar wave vectors and their amplitudes
    def aggregate_amplitudes_sample(H_sample):
        wave_vector_sums = {}
        for wave_vector, amplitude in H_sample:
            rounded_wave_vector = np.round(wave_vector,1)
            if rounded_wave_vector in wave_vector_sums:
                wave_vector_sums[rounded_wave_vector] += amplitude
            else:
                wave_vector_sums[rounded_wave_vector] = amplitude
        H_sample_reduced = [(wave_vector, amplitude) for wave_vector, amplitude in wave_vector_sums.items()]
        return H_sample_reduced

    # Get H_reduced
    H_sample_reduced = aggregate_amplitudes_sample(H)

    # Find max amplitude in the region of 0.9k_m to 1.1k_m and its wave vector
    def find_max_amplitude_and_wave_vector(H_sample_reduced, k_m):
        max_ampl = 0
        k_max = 0
        for wave_vector, amplitude in H_sample_reduced:
            if 0.9 * k_m <= wave_vector <= 1.1 * k_m:
                if amplitude > max_ampl:
                    max_ampl = amplitude
                    k_max = wave_vector
        return max_ampl, k_max

    # Find max amplitude at zero wave vector
    def find_H0_sample(H_sample_reduced):
        H_0_sample = 0
        for wave_vector, amplitude in H_sample_reduced:
            if wave_vector == 0:
                H_0_sample = amplitude
                break
        return H_0_sample

    # Calculate Max_ampl, K_max, and H_0
    Max_ampl, K_max = find_max_amplitude_and_wave_vector(H_sample_reduced, k_m)
    H_0_sample = find_H0_sample(H_sample_reduced)

    # Calculate sinc functions for the slit size
    sinc_km = np.sinc(k_m * slit_size / (2 * pi))
    sinc_H0_sample = np.sinc(H_0_sample * slit_size / (2 * pi)) if H_0_sample != 0 else 0

    # Calculate contrast V_sample
    V_sample = np.abs(2 * Max_ampl * sinc_km / (H_0_sample * sinc_H0_sample)) if H_0_sample != 0 else None
    return V_sample, K_max

# Range for d
d_range = np.arange(0.001, 0.085, 0.001)  # Range for d values

# Create dictionary to store V_sample for different d values
V_sample_dict = {}

# Store results to avoid recalculating
results_dict = {lam: [calculate_contrast_s_and_kmax(d, lam) for d in d_range] for lam in lambdas}

# Store V_sample in the dictionary, ignoring values below 1% contrast
for lam in lambdas:
    V_sample_dict[lam] = {d: calculate_contrast_s_and_kmax(d, lam)[0] for d in d_range if calculate_contrast_s_and_kmax(d, lam)[0] >= 0.01}

# Print the V_sample dictionary
print("V_sample dictionary:")
for lam, d_values in V_sample_dict.items():
    print(f"\nFor wavelength λ = {lam:.1e} m:")
    for d, V_sample in d_values.items():
        print(f"d = {d:.4f} m, V_sample = {V_sample:.4f}")

# Save results to a CSV file, only including values above the 1% threshold
csv_filename = 'V_withsample_phi_filtered.csv'
contrast_data = []

for lam in lambdas:
    for d in d_range:
        V_sample, K_max = calculate_contrast_s_and_kmax(d, lam)
        if V_sample >= 0.01:
            contrast_data.append([d, V_sample])

with open(csv_filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['d (m)', 'V_sample'])
    csvwriter.writerows(contrast_data)

print(f'Results saved to {csv_filename}')

# Plot Contrast V_sample vs d for different wavelengths
fig, ax1 = plt.subplots(figsize=(10, 6))

for lam in lambdas:
    # Retrieve results for current lambda
    contrast_sample_values, kmax_values = zip(*[(V_sample, K_max) for V_sample, K_max in results_dict[lam] if V_sample >= 0.01])

    # Plot contrast vs d on the primary y-axis
    ax1.plot(d_range[:len(contrast_sample_values)], contrast_sample_values, marker='o', linestyle='-', label=f'Contrast V_sample (λ={lam:.1e} m)')

# Add labels and legend
ax1.set_xlabel('d (meters)')
ax1.set_ylabel('Contrast V_sample')
ax1.set_title('Sample_added-Contrast V_sample vs d for monochromatic')
ax1.legend(loc='upper right')
ax1.grid(True)

# Create a secondary y-axis for K_max
ax2 = ax1.twinx()

for lam in lambdas:
    # Retrieve results for current lambda
    contrast_values, kmax_values = zip(*[(V_sample, K_max) for V_sample, K_max in results_dict[lam] if V_sample >= 0.01])

    # Plot K_max vs d on the secondary y-axis
    # ax2.plot(d_range[:len(kmax_values)], kmax_values, marker='s', linestyle='--', label=f'K_max (λ={lam:.1e} m)')

# Add labels and legend for the secondary y-axis
ax2.set_ylabel('K_max')
ax2.legend(loc='upper left')

plt.show()
#################################.  V_ratio ####################################################
#################################.  V_ratio ####################################################
#################################.  V_ratio ####################################################
#################################.  V_ratio ####################################################
#################################.  V_ratio ####################################################
#################################.  V_ratio ####################################################
#################################.  V_ratio ####################################################
#################################.  V_ratio ####################################################
#################################.  V_ratio ####################################################
#################################.  V_ratio ####################################################
###############$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$       V_ratio      $$$$$$$$$$$$$$$$$$$$$$$$$
# Calculate V_sample/V and store in a new dictionary
import csv
import matplotlib.pyplot as plt

# Assuming V_dict_filtered and V_sample_dict are already defined and populated

# Calculate V_ratio only for valid values in both V_dict_filtered and V_sample_dict
V_ratio_dict = {}

for lam in lambdas:
    V_ratio_dict[lam] = {}
    for d in V_sample_dict[lam].keys():
        if d in V_dict_filtered[lam]:
            V_ratio = V_sample_dict[lam][d] / V_dict_filtered[lam][d]
            V_ratio_dict[lam][d] = V_ratio

# Print the V_ratio dictionary
print("V_ratio dictionary:")
for lam, d_values in V_ratio_dict.items():
    print(f"\nFor wavelength λ = {lam:.1e} m:")
    for d, V_ratio in d_values.items():
        print(f"d = {d:.4f} m, V_ratio = {V_ratio:.4f}")

# Save the V_ratio results to a CSV file
filename = 'V_ratio_filtered.csv'
with open(filename, 'w', newline='') as csvfile:
    fieldnames = ['d (m)', 'V_ratio']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for lam, d_values in V_ratio_dict.items():
        for d, V_ratio in d_values.items():
            writer.writerow({'d (m)': d, 'V_ratio': round(V_ratio, 4)})  # Rounded to 4 decimal places

# Plot V_ratio vs d for different wavelengths
fig, ax = plt.subplots(figsize=(10, 6))

for lam in lambdas:
    d_values = list(V_ratio_dict[lam].keys())
    V_ratio_values = list(V_ratio_dict[lam].values())
    
    # Plot V_ratio vs d
    ax.plot(d_values, V_ratio_values, marker='o', linestyle='-', label=f'V_ratio (λ={lam:.1e} m)')

# Add labels and legend
ax.set_xlabel('d (meters)')
ax.set_ylabel('V_ratio')
ax.set_title('V_ratio vs d for different wavelengths')
ax.legend(loc='upper right')
ax.grid(True)
ax.set_ylim(bottom=0, top=1)
plt.show()

